# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# Шаги:
* Открыть файл с данными и изучить его.
* Разделить исходные данные на обучающую, валидационную и тестовую выборки.
* Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.
* Проверить качество модели на тестовой выборке.
* Дополнительное задание: проверить модель на вменяемость.

## Откройте и изучите файл

Для начала импортируем нужные библиотеки и функции и рассмотрим файл в общих чертах. Проверим, всё ли в порядке с данными. 

In [1]:
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Пропусков нет, форматы данных корректны, идём дальше. 

## Разбейте данные на выборки

Разобьём данные на 3 выборки: обучающую, валидационную и тестовую. 

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [7]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid,test_size = 0.5, 
                                     random_state=12345)

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
<font color='4682B4'>
Соотношение обучающей, тестовой и валидационной выборок составляет 3:1:1.
 Обучающая(60%) и валидационная(40%) выборки образуются в результате деления общей, далее валидационная ещё делится на 2, и в итоге получаются 3 выборки:
    обучающая(60%), валидационная(20%) и тестовая(20%).</font>
</div>

In [8]:
print('Размер обучающей выборки:', df_train.shape[0],'объектов')
print('Размер валидационной выборки:', df_valid.shape[0],'объекта')
print('Размер тестовой выборки:',df_test.shape[0],'объекта')

Размер обучающей выборки: 1928 объектов
Размер валидационной выборки: 643 объекта
Размер тестовой выборки: 643 объекта


In [9]:
# Набор данных обучающей выборки:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
# Набор данных тестовой выборки:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']
# Набор данных валидационной выборки:
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

## Исследуйте модели

In [10]:
names = []
results = []

Обучим модель 'Дерево Решений' и найдём лучшую глубину

In [11]:
best_result = 0
best_depth = 0
best_model = None
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_depth = depth
        best_model = model
        best_result = result
print(f'глубина: {best_depth}, результат: {best_result}')
names.append('DecisionTreeClassifier')
results.append(best_result)

глубина: 3, результат: 0.7853810264385692


Обучим модель 'Cлучайный лес' и найдём лучшее количество деревьев и глубину.

In [12]:
for est in range(15,35):
    for depth in range(1,15):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, 
                                       n_estimators=est)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_depth = depth
            best_model = model
            best_result = result
            best_est = est
print(f'количество деревьев: {best_est}, глубина: {best_depth},\
результат: {best_result}')
names.append('RandomForestClassifier')
results.append(best_result)

количество деревьев: 26, глубина: 7,результат: 0.807153965785381


Обучим модель 'Логистическая регрессия'

In [13]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = model.score(features_valid, target_valid)
print(result)
names.append('LogisticRegression')
results.append(result)

0.7107309486780715


In [14]:
pd.DataFrame(results, names)

,0
DecisionTreeClassifier,0.785381
RandomForestClassifier,0.807154
LogisticRegression,0.710731


* Точность модели 'Случайный лес' при 26 деревьях и глубиной 7 - 0.8. Именно её мы и проверим на тестовой выборке. 
* За ней идёт модель 'Дерево решений' с глубиной 3 - 0.785
* На последнем месте располагается модель 'Логистическая регрессия' - 0.71

## Проверьте модель на тестовой выборке

Проверим модель 'Случайный лес' на тестовой выборке при 26 деревьях и глубине 7

In [15]:
model = RandomForestClassifier(random_state=12345, max_depth=7, n_estimators=26)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
score = accuracy_score(target_test, predictions)
print('Accuracy:', score)

Accuracy: 0.80248833592535


Модель показала accuracy = 0.8, что больше 0.75

## (бонус) Проверьте модели на адекватность

Чтобы проверить нашу модель на адекватность, посмотрим не предсказывает ли она только 0 или 1. Для этого взглянем на доли двух тарифов.

In [16]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Проверка модели на адекватность пройдена. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


# Общий вывод:
* Наша модель с вероятностью 80% подобирает подходящий тарифный план.
* Наилучшей оказалась модель 'Случайный лес', которая показала точность тестовой модели = 0.8, а так же прошла проверку на вменяемость.
* Точность модели 'Случайный лес' при 26 деревьях и глубиной 7 - 0.8. Именно её мы и проверили на тестовой выборке. 
* За ней идёт модель 'Дерево решений' с глубиной 3 - 0.785
* На последнем месте располагается модель 'Логистическая регрессия' - 0.71